# Imports

In [64]:
import os
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

SPOTIPY_CLIENT_ID = ''
SPOTIPY_CLIENT_SECRET = ''
SPOTIPY_REDIRECT_URI = ''

In [48]:
username = ''
scope = 'playlist-modify-public'

token = util.prompt_for_user_token(username,scope,
                                   client_id= SPOTIPY_CLIENT_ID,
                                   client_secret = SPOTIPY_CLIENT_SECRET,
                                   redirect_uri = SPOTIPY_REDIRECT_URI)
sp = spotipy.Spotify(auth=token)

# Import data from Every Noise at Once playlists

In [3]:
# The Spotify API does not provide genres for the majority of tracks and albums, so I will be using the auto-generated
# playlists created by Every Noise at Once (everynoise.com) to populate a dataframe with information.

In [23]:
# define Spotify playlist ids
hip_hop_playlist = '6MXkE0uYF4XwU4VTtyrpfP'
pop_playlist = '6gS3HhOiI17QNojjPuPzqc'
rock_playlist = '7dowgSWOmvdpwNkGFMUs6e'
trap_playlist = '60SHtDyagDjPnUpC7x1UD9'
edm_playlist = '3pDxuMpz94eDs7WFqudTbZ'

# create empty track id lists for each playlist
hip_hop_tracks = []
pop_tracks = []
rock_tracks = []
trap_tracks = []
edm_tracks = []

# query Spotify for information about each track in each playlist (limit of 100 tracks each)
hip_hop_info = sp.playlist_tracks(hip_hop_playlist)['items']
pop_info = sp.playlist_tracks(pop_playlist)['items']
rock_info = sp.playlist_tracks(rock_playlist)['items']
trap_info = sp.playlist_tracks(trap_playlist)['items']
edm_info = sp.playlist_tracks(edm_playlist)['items']

# iterate through queries and add track ids to lists created above

for idx,item in enumerate(hip_hop_info):
    hip_hop_tracks.append(item['track']['id'])
    
for idx,item in enumerate(pop_info):
    pop_tracks.append(item['track']['id'])
    
for idx,item in enumerate(rock_info):
    rock_tracks.append(item['track']['id'])
    
for idx,item in enumerate(trap_info):
    trap_tracks.append(item['track']['id'])
    
for idx,item in enumerate(edm_info):
    edm_tracks.append(item['track']['id'])

In [ ]:
# create empty lists for each audio feature available from Spotify for each playlist
# audio features: danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness,
# valence, tempo, time_signature

In [61]:
# hip hop
hip_hop_danceability = []
hip_hop_energy = []
hip_hop_key = []
hip_hop_loudness = []
hip_hop_mode = []
hip_hop_speechiness = []
hip_hop_acousticness = []
hip_hop_instrumentalness = []
hip_hop_liveness = []
hip_hop_valence = []
hip_hop_tempo = []
hip_hop_time_signature = []

# pop
pop_danceability = []
pop_energy = []
pop_key = []
pop_loudness = []
pop_mode = []
pop_speechiness = []
pop_acousticness = []
pop_instrumentalness = []
pop_liveness = []
pop_valence = []
pop_tempo = []
pop_time_signature = []

# rock
rock_danceability = []
rock_energy = []
rock_key = []
rock_loudness = []
rock_mode = []
rock_speechiness = []
rock_acousticness = []
rock_instrumentalness = []
rock_liveness = []
rock_valence = []
rock_tempo = []
rock_time_signature = []

# trap
trap_danceability = []
trap_energy = []
trap_key = []
trap_loudness = []
trap_mode = []
trap_speechiness = []
trap_acousticness = []
trap_instrumentalness = []
trap_liveness = []
trap_valence = []
trap_tempo = []
trap_time_signature = []

# edm
edm_danceability = []
edm_energy = []
edm_key = []
edm_loudness = []
edm_mode = []
edm_speechiness = []
edm_acousticness = []
edm_instrumentalness = []
edm_liveness = []
edm_valence = []
edm_tempo = []
edm_time_signature = []

In [62]:
# iterate through the hip hop track list and append feature values to respective lists
hip_hop_features = sp.audio_features(hip_hop_tracks)
for idx,id in enumerate(hip_hop_tracks):
    hip_hop_danceability.append(hip_hop_features[idx]['danceability'])
    hip_hop_energy.append(hip_hop_features[idx]['energy'])
    hip_hop_key.append(hip_hop_features[idx]['key'])
    hip_hop_loudness.append(hip_hop_features[idx]['loudness'])
    hip_hop_mode.append(hip_hop_features[idx]['mode'])
    hip_hop_speechiness.append(hip_hop_features[idx]['speechiness'])
    hip_hop_acousticness.append(hip_hop_features[idx]['acousticness'])
    hip_hop_instrumentalness.append(hip_hop_features[idx]['instrumentalness'])
    hip_hop_liveness.append(hip_hop_features[idx]['liveness'])
    hip_hop_valence.append(hip_hop_features[idx]['valence'])
    hip_hop_tempo.append(hip_hop_features[idx]['tempo'])
    hip_hop_time_signature.append(hip_hop_features[idx]['time_signature'])

# iterate through the pop track list and append feature values to respective lists
pop_features = sp.audio_features(pop_tracks)
for idx,id in enumerate(pop_tracks):
    pop_danceability.append(pop_features[idx]['danceability'])
    pop_energy.append(pop_features[idx]['energy'])
    pop_key.append(pop_features[idx]['key'])
    pop_loudness.append(pop_features[idx]['loudness'])
    pop_mode.append(pop_features[idx]['mode'])
    pop_speechiness.append(pop_features[idx]['speechiness'])
    pop_acousticness.append(pop_features[idx]['acousticness'])
    pop_instrumentalness.append(pop_features[idx]['instrumentalness'])
    pop_liveness.append(pop_features[idx]['liveness'])
    pop_valence.append(pop_features[idx]['valence'])
    pop_tempo.append(pop_features[idx]['tempo'])
    pop_time_signature.append(pop_features[idx]['time_signature'])

# iterate through the rock track list and append feature values to respective lists
rock_features = sp.audio_features(rock_tracks)
for idx,id in enumerate(rock_tracks):
    rock_danceability.append(rock_features[idx]['danceability'])
    rock_energy.append(rock_features[idx]['energy'])
    rock_key.append(rock_features[idx]['key'])
    rock_loudness.append(rock_features[idx]['loudness'])
    rock_mode.append(rock_features[idx]['mode'])
    rock_speechiness.append(rock_features[idx]['speechiness'])
    rock_acousticness.append(rock_features[idx]['acousticness'])
    rock_instrumentalness.append(rock_features[idx]['instrumentalness'])
    rock_liveness.append(rock_features[idx]['liveness'])
    rock_valence.append(rock_features[idx]['valence'])
    rock_tempo.append(rock_features[idx]['tempo'])
    rock_time_signature.append(rock_features[idx]['time_signature'])

# iterate through the trap track list and append feature values to respective lists
trap_features = sp.audio_features(trap_tracks)
for idx,id in enumerate(trap_tracks):
    trap_danceability.append(trap_features[idx]['danceability'])
    trap_energy.append(trap_features[idx]['energy'])
    trap_key.append(trap_features[idx]['key'])
    trap_loudness.append(trap_features[idx]['loudness'])
    trap_mode.append(trap_features[idx]['mode'])
    trap_speechiness.append(trap_features[idx]['speechiness'])
    trap_acousticness.append(trap_features[idx]['acousticness'])
    trap_instrumentalness.append(trap_features[idx]['instrumentalness'])
    trap_liveness.append(trap_features[idx]['liveness'])
    trap_valence.append(trap_features[idx]['valence'])
    trap_tempo.append(trap_features[idx]['tempo'])
    trap_time_signature.append(trap_features[idx]['time_signature'])

# iterate through the edm track list and append feature values to respective lists
edm_features = sp.audio_features(edm_tracks)
for idx,id in enumerate(edm_tracks):
    edm_danceability.append(edm_features[idx]['danceability'])
    edm_energy.append(edm_features[idx]['energy'])
    edm_key.append(edm_features[idx]['key'])
    edm_loudness.append(edm_features[idx]['loudness'])
    edm_mode.append(edm_features[idx]['mode'])
    edm_speechiness.append(edm_features[idx]['speechiness'])
    edm_acousticness.append(edm_features[idx]['acousticness'])
    edm_instrumentalness.append(edm_features[idx]['instrumentalness'])
    edm_liveness.append(edm_features[idx]['liveness'])
    edm_valence.append(edm_features[idx]['valence'])
    edm_tempo.append(edm_features[idx]['tempo'])
    edm_time_signature.append(edm_features[idx]['time_signature'])

In [73]:
hip_hop_df = pd.DataFrame(data=zip(hip_hop_danceability,
                                   hip_hop_energy,
                                   hip_hop_key,
                                   hip_hop_loudness,
                                   hip_hop_mode,
                                   hip_hop_speechiness,
                                   hip_hop_acousticness,
                                   hip_hop_instrumentalness,
                                   hip_hop_liveness,
                                   hip_hop_valence,
                                   hip_hop_tempo,
                                   hip_hop_time_signature),
                         index=hip_hop_tracks,
                         columns=['danceability',
                                  'energy',
                                  'key',
                                  'loudness',
                                  'mode',
                                  'speechiness',
                                  'acousticness',
                                  'instrumentalness',
                                  'liveness',
                                  'valence',
                                  'tempo',
                                  'time_signature'])
hip_hop_df['hip_hop'] = 1

In [74]:
pop_df = pd.DataFrame(data=zip(pop_danceability,
                                   pop_energy,
                                   pop_key,
                                   pop_loudness,
                                   pop_mode,
                                   pop_speechiness,
                                   pop_acousticness,
                                   pop_instrumentalness,
                                   pop_liveness,
                                   pop_valence,
                                   pop_tempo,
                                   pop_time_signature),
                         index=pop_tracks,
                         columns=['danceability',
                                  'energy',
                                  'key',
                                  'loudness',
                                  'mode',
                                  'speechiness',
                                  'acousticness',
                                  'instrumentalness',
                                  'liveness',
                                  'valence',
                                  'tempo',
                                  'time_signature'])
pop_df['pop'] = 1

In [75]:
rock_df = pd.DataFrame(data=zip(rock_danceability,
                                   rock_energy,
                                   rock_key,
                                   rock_loudness,
                                   rock_mode,
                                   rock_speechiness,
                                   rock_acousticness,
                                   rock_instrumentalness,
                                   rock_liveness,
                                   rock_valence,
                                   rock_tempo,
                                   rock_time_signature),
                         index=rock_tracks,
                         columns=['danceability',
                                  'energy',
                                  'key',
                                  'loudness',
                                  'mode',
                                  'speechiness',
                                  'acousticness',
                                  'instrumentalness',
                                  'liveness',
                                  'valence',
                                  'tempo',
                                  'time_signature'])
rock_df['rock'] = 1

In [76]:
trap_df = pd.DataFrame(data=zip(trap_danceability,
                                   trap_energy,
                                   trap_key,
                                   trap_loudness,
                                   trap_mode,
                                   trap_speechiness,
                                   trap_acousticness,
                                   trap_instrumentalness,
                                   trap_liveness,
                                   trap_valence,
                                   trap_tempo,
                                   trap_time_signature),
                         index=trap_tracks,
                         columns=['danceability',
                                  'energy',
                                  'key',
                                  'loudness',
                                  'mode',
                                  'speechiness',
                                  'acousticness',
                                  'instrumentalness',
                                  'liveness',
                                  'valence',
                                  'tempo',
                                  'time_signature'])
trap_df['trap'] = 1

In [77]:
edm_df = pd.DataFrame(data=zip(edm_danceability,
                                   edm_energy,
                                   edm_key,
                                   edm_loudness,
                                   edm_mode,
                                   edm_speechiness,
                                   edm_acousticness,
                                   edm_instrumentalness,
                                   edm_liveness,
                                   edm_valence,
                                   edm_tempo,
                                   edm_time_signature),
                         index=edm_tracks,
                         columns=['danceability',
                                  'energy',
                                  'key',
                                  'loudness',
                                  'mode',
                                  'speechiness',
                                  'acousticness',
                                  'instrumentalness',
                                  'liveness',
                                  'valence',
                                  'tempo',
                                  'time_signature'])
edm_df['edm'] = 1

In [90]:
combined_df = pd.concat(objs=[hip_hop_df,pop_df,rock_df,trap_df,edm_df])

C:\Users\Joe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [91]:
combined_df = combined_df.replace(to_replace=np.nan, value=0)

In [92]:
combined_df = combined_df.rename_axis('track_id')

In [93]:
combined_df.head()

,acousticness,danceability,edm,energy,hip_hop,instrumentalness,key,liveness,loudness,mode,pop,rock,speechiness,tempo,time_signature,trap,valence
track_id,,,,,,,,,,,,,,,,,
3Ti0GdlrotgwsAVBBugv0I,0.17300,0.848,0.0,0.666,1.0,0.000699,0,0.1290,-6.547,1,0.0,0.0,0.274,96.662,4,0.0,0.744
1w29UTa5uUvIri2tWtZ12Y,0.10200,0.862,0.0,0.642,1.0,0.000000,6,0.2720,-13.652,0,0.0,0.0,0.101,99.201,4,0.0,0.765
30oTS7bm0aH3p7lqjEIu8q,0.02270,0.875,0.0,0.745,1.0,0.000002,1,0.2870,-5.878,1,0.0,0.0,0.269,95.201,4,0.0,0.749
0IVAlb0cFCH1cQ974iR1TU,0.14800,0.782,0.0,0.589,1.0,0.000000,1,0.1640,-4.827,1,0.0,0.0,0.189,90.438,4,0.0,0.869
48ZUwXrEOhaXLCxvmRYhZv,0.00556,0.737,0.0,0.890,1.0,0.001710,4,0.0801,-8.443,1,0.0,0.0,0.158,105.848,4,0.0,0.809


In [94]:
combined_df['hip_hop'] = combined_df['hip_hop'].astype(int)
combined_df['pop'] = combined_df['pop'].astype(int)
combined_df['rock'] = combined_df['rock'].astype(int)
combined_df['trap'] = combined_df['trap'].astype(int)
combined_df['edm'] = combined_df['edm'].astype(int)

# Exploratory Analysis